In [1]:
from datetime import datetime as dt
from itertools import chain

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from torch.utils.tensorboard import SummaryWriter

# Load and preprocess the dataset (assuming you have a CSV file)
df = pd.read_csv("../data/diamonds.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [2]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [3]:
cat_columns = ["cut", "color", "clarity"]
num_columns = ["carat", "depth", "table", "x", "y", "z"]
cat_values = pd.unique(df[cat_columns].values.ravel("K"))
target_column = "price"
tokens = list(
    chain(
        cat_values,
        cat_columns,
        num_columns,
        ["PAD", "[NUMERIC_MASK]", "[MASK]"],
        [target_column],
    )
)
token_dict = {token: i for i, token in enumerate(tokens)}

In [4]:
embedding = nn.Embedding(len(token_dict), 64)
cat_values_emb = torch.tensor(
    [[token_dict[token] for token in row] for row in df[cat_columns].values],
    dtype=torch.long,
)

col_names_emb = torch.tensor([token_dict[col] for col in df.columns], dtype=torch.long)
embedding(col_names_emb).shape

torch.Size([10, 64])

In [5]:
X = df.drop("price", axis=1)
y = df["price"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Preprocess categorical features
X_train_cat = X_train[cat_columns].copy()
X_test_cat = X_test[cat_columns].copy()

label_encoders = {}
for col in cat_columns:
    le = LabelEncoder()
    X_train_cat[col] = X_train_cat[col].map(token_dict)
    X_test_cat[col] = X_test_cat[col].map(token_dict)
    # label_encoders[col] = le

# Preprocess numeric features

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train[num_columns].copy())
X_test_num = scaler.transform(X_test[num_columns].copy())

X_train_cat_tensor = torch.tensor(
    X_train_cat.values, dtype=torch.int64
)  # Use int64 dtype for categorical indices
X_train_num_tensor = torch.tensor(X_train_num, dtype=torch.float32)
X_test_cat_tensor = torch.tensor(
    X_test_cat.values, dtype=torch.int64
)  # Use int64 dtype for categorical indices
X_test_num_tensor = torch.tensor(X_test_num, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

In [6]:
X_test_cat

,cut,color,clarity
1388,0,10,17
50052,3,9,16
41645,0,5,16
42377,1,5,16
17244,0,5,12
...,...,...,...
44081,3,5,14
23713,3,9,16
31375,2,10,17
21772,0,9,14


In [7]:
input_tensor = torch.randn(10, 6)
repeated_tensor = input_tensor.unsqueeze(1).repeat(1, 4, 1)

print(repeated_tensor.shape)

torch.Size([10, 4, 6])


In [8]:
X_train_num_tensor[0:10, :].unsqueeze(2).shape

torch.Size([10, 6, 1])

In [9]:
test_mat = torch.rand(3, 4, 5)
test_mat

tensor([[[0.7776, 0.7208, 0.2569, 0.9848, 0.0419],
         [0.2441, 0.9367, 0.3139, 0.6594, 0.9689],
         [0.8419, 0.5623, 0.2690, 0.3838, 0.9350],
         [0.6744, 0.7276, 0.9973, 0.8898, 0.5496]],

        [[0.2172, 0.0157, 0.0079, 0.3110, 0.5779],
         [0.7725, 0.7939, 0.3447, 0.8620, 0.8768],
         [0.7157, 0.4711, 0.1653, 0.6591, 0.9378],
         [0.9363, 0.1349, 0.0369, 0.6946, 0.6196]],

        [[0.4562, 0.2732, 0.0817, 0.3020, 0.6203],
         [0.7903, 0.6836, 0.0828, 0.1662, 0.7791],
         [0.8608, 0.5242, 0.7859, 0.2603, 0.6366],
         [0.4818, 0.2743, 0.3332, 0.2533, 0.3633]]])

In [10]:
test_mat[0, 0], test_mat[0, 0] * 10

(tensor([0.7776, 0.7208, 0.2569, 0.9848, 0.0419]),
 tensor([7.7759, 7.2082, 2.5692, 9.8482, 0.4194]))

In [11]:
(X_train_num_tensor[0:10, :].shape, X_train_num_tensor[0:10, :].unsqueeze(2).shape)

(torch.Size([10, 6]), torch.Size([10, 6, 1]))

In [12]:
scalars = torch.tensor([10, 2, 3, 4])
scalars = scalars.unsqueeze(1).unsqueeze(0)
scalars.shape

torch.Size([1, 4, 1])

In [13]:
double_test_mat = scalars * test_mat
double_test_mat

tensor([[[7.7759, 7.2082, 2.5692, 9.8482, 0.4194],
         [0.4881, 1.8734, 0.6278, 1.3188, 1.9378],
         [2.5258, 1.6870, 0.8071, 1.1515, 2.8049],
         [2.6975, 2.9105, 3.9892, 3.5592, 2.1983]],

        [[2.1718, 0.1568, 0.0792, 3.1099, 5.7787],
         [1.5450, 1.5878, 0.6894, 1.7239, 1.7537],
         [2.1471, 1.4133, 0.4958, 1.9772, 2.8133],
         [3.7453, 0.5396, 0.1475, 2.7782, 2.4785]],

        [[4.5616, 2.7319, 0.8165, 3.0202, 6.2032],
         [1.5806, 1.3672, 0.1655, 0.3325, 1.5583],
         [2.5825, 1.5727, 2.3576, 0.7808, 1.9099],
         [1.9274, 1.0972, 1.3329, 1.0130, 1.4533]]])

In [14]:
double_test_mat[0, 0]

tensor([7.7759, 7.2082, 2.5692, 9.8482, 0.4194])

In [15]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_head = d_model // n_heads

        self.q_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.out_linear = nn.Linear(d_model, d_model)

    def forward(self, q, k, v, mask=None):
        batch_size = q.size(0)

        q = (
            self.q_linear(q)
            .view(batch_size, -1, self.n_heads, self.d_head)
            .transpose(1, 2)
        )
        k = (
            self.k_linear(k)
            .view(batch_size, -1, self.n_heads, self.d_head)
            .transpose(1, 2)
        )
        v = (
            self.v_linear(v)
            .view(batch_size, -1, self.n_heads, self.d_head)
            .transpose(1, 2)
        )

        attn_output, _ = self.scaled_dot_product_attention(q, k, v, mask)

        attn_output = (
            attn_output.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        )
        out = self.out_linear(attn_output)
        return out

    def scaled_dot_product_attention(self, q, k, v, mask=None):
        matmul_qk = torch.matmul(q, k.transpose(-2, -1))
        d_k = q.size(-1)
        scaled_attention_logits = matmul_qk / (d_k**0.5)

        if mask is not None:
            scaled_attention_logits += mask * -1e9

        attention_weights = F.softmax(scaled_attention_logits, dim=-1)
        output = torch.matmul(attention_weights, v)

        return output, attention_weights


class TransformerEncoderLayer(nn.Module):
    def __init__(self, d_model, n_heads):
        super(TransformerEncoderLayer, self).__init__()

        self.multi_head_attention = MultiHeadAttention(d_model, n_heads)

        self.feed_forward = nn.Sequential(
            nn.Linear(d_model, 4 * d_model),
            nn.ReLU(),
            nn.Linear(4 * d_model, d_model),
        )

        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)

    def forward(self, q, k, v, mask=None):
        attn_output = self.multi_head_attention(q, k, v, mask)
        out1 = self.layernorm1(q + attn_output)

        ff_output = self.feed_forward(out1)
        out2 = self.layernorm2(out1 + ff_output)

        return out2


# Parameters
d_model = 64  # Embedding dimension
n_heads = 4  # Number of attention heads
seq_len_q = 10  # Sequence length for the query tensor
seq_len_k = 20  # Sequence length for the key tensor
batch_size = 32  # Batch size

# Random data
q = torch.rand((batch_size, seq_len_q, d_model))
k = torch.rand((batch_size, seq_len_k, d_model))
v = k  # Usually, value and key are the same in many applications

# Model
encoder_layer = TransformerEncoderLayer(d_model, n_heads)

# Forward pass
output = encoder_layer(q, k, v)
print("Output shape:", output.shape)

Output shape: torch.Size([32, 10, 64])


In [16]:
torch.tensor(1)

tensor(1)

In [17]:
class TabTransformer(nn.Module):
    def __init__(
        self,
        tokens,
        numeric_col_tokens,
        cat_col_tokens,
        token_dict,
        d_model=128,
        n_heads=8,
    ):
        super(TabTransformer, self).__init__()
        self.d_model = d_model
        self.tokens = tokens
        self.token_dict = token_dict
        # Masks
        self.cat_mask_token = torch.tensor(self.token_dict["[MASK]"])
        self.numeric_mask_token = torch.tensor(self.token_dict["[NUMERIC_MASK]"])

        self.col_tokens = cat_col_tokens + numeric_col_tokens
        self.n_tokens = len(tokens)  # TODO Make this
        # Embedding layers for categorical features
        self.embeddings = nn.Embedding(self.n_tokens, self.d_model)
        self.n_numeric_cols = len(numeric_col_tokens)
        self.n_cat_cols = len(cat_col_tokens)
        self.n_columns = self.n_numeric_cols + self.n_cat_cols
        # self.numeric_embeddings = NumericEmbedding(d_model=self.d_model)
        self.col_indices = torch.tensor(
            [self.tokens.index(col) for col in self.col_tokens], dtype=torch.long
        )
        self.numeric_indices = torch.tensor(
            [self.tokens.index(col) for col in numeric_col_tokens], dtype=torch.long
        )
        self.transformer_encoder = TransformerEncoderLayer(d_model, n_heads=n_heads)

        self.regressor = nn.Sequential(
            nn.Linear(d_model, d_model * 2),
            nn.ReLU(),
            nn.Linear(d_model * 2, 1),
            nn.ReLU(),
        )

        self.mlm_decoder = nn.Sequential(
            nn.Linear(d_model, d_model)
        )  # TODO try making more complex

        self.mnm_decoder = nn.Sequential(
            nn.Linear(self.n_columns * self.d_model, 128),  # Try making more complex
            nn.ReLU(),
            nn.Linear(128, 6),
        )

        self.flatten_layer = nn.Linear(len(self.col_tokens), 1)

    def forward(self, num_inputs, cat_inputs, mnm=None, mlm=None, task="regression"):
        # Embed column indices
        repeated_col_indices = self.col_indices.unsqueeze(0).repeat(
            num_inputs.size(0), 1
        )
        col_embeddings = self.embeddings(repeated_col_indices)

        repeated_numeric_indices = self.numeric_indices.unsqueeze(0).repeat(
            num_inputs.size(0), 1
        )
        numeric_col_embeddings = self.embeddings(repeated_numeric_indices)

        if mlm is not None:
            # print(f"cat_inputs.shape: {cat_inputs.shape}, mlm.shape: {mlm.shape}")
            cat_inputs.masked_fill_(mlm, self.cat_mask_token)  # `_` is inplace.

        cat_embeddings = self.embeddings(cat_inputs)
        expanded_num_inputs = num_inputs.unsqueeze(2).repeat(1, 1, self.d_model)
        num_embeddings = numeric_col_embeddings * expanded_num_inputs

        if mnm is not None:
            # print(
            #     f"mnm.shape: {mnm.shape}, num_embeddings.shape: {num_embeddings.shape}"
            # )
            # print(num_embeddings.sum())
            numeric_mask_embedding = self.embeddings(self.numeric_mask_token)
            # numeric_mask_embedding = torch.ones_like(numeric_mask_embedding)
            # numeric_mask_embedding = torch.zeros_like(numeric_mask_embedding)
            num_embeddings[mnm] = numeric_mask_embedding
            # print(num_embeddings.sum())

        query_embeddings = torch.cat([cat_embeddings, num_embeddings], dim=1)
        out = self.transformer_encoder(
            col_embeddings,
            query_embeddings,
            query_embeddings
            # col_embeddings, query_embeddings, query_embeddings
        )
        if task == "regression":
            out = self.regressor(out)
            out = self.flatten_layer(out.squeeze(-1))

            return out
        elif task == "mlm":
            cat_out = self.mlm_decoder(out)
            # print(f"Out shape: {out.shape}, cat_out shape: {cat_out.shape}")
            numeric_out = out.view(out.size(0), -1)
            # print(f"numeric_out shape: {numeric_out.shape}")
            numeric_out = self.mnm_decoder(numeric_out)
            return cat_out, numeric_out
        else:
            raise ValueError(f"Task {task} not supported.")


no_price_tokens = tokens.copy()
no_price_tokens.remove("price")

numeric_col_tokens = (
    df.head().drop("price", axis=1).select_dtypes(include=np.number).columns.to_list()
)
cat_col_tokens = df.head().select_dtypes(exclude=np.number).columns.to_list()

model = TabTransformer(
    no_price_tokens,
    numeric_col_tokens=numeric_col_tokens,
    cat_col_tokens=cat_col_tokens,
    token_dict=token_dict,
)
batch_size = 3
mnm = torch.rand(batch_size, X_train_num_tensor.size(1)) > 0.8
mlm = torch.rand(batch_size, X_train_cat_tensor.size(1)) > 0.8
with torch.no_grad():
    x = model(
        X_train_num_tensor[0:batch_size, :],
        X_train_cat_tensor[0:batch_size, :],
        mnm,
        mlm,
        task="mlm",
    )
x[0].shape, x[1].shape

RuntimeError: shape '[3, -1, 6, 21]' is invalid for input of size 3456

In [ ]:
# Masked Tabualr Modeling
epochs = 20
batch_size = 1000
lr = 0.001
mse_loss = nn.MSELoss()
ce_loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

model_time = dt.now()
model_time = model_time.strftime("%Y-%m-%dT%H:%M:%S")
model_name = f"TryAgainPretrain.8_{model_time}"

summary_writer = SummaryWriter("runs/" + model_name)

batch_count = 0
model.train()
for epoch in range(epochs):
    for i in range(0, X_train_num_tensor.size(0), batch_size):
        num_inputs = X_train_num_tensor[i : i + batch_size, :]
        cat_inputs = X_train_cat_tensor[i : i + batch_size, :]
        mlm = torch.rand(cat_inputs.shape) > 0.8  # Greater than 1 for testing
        mnm = torch.rand(num_inputs.shape) > 0.8
        # mlm = None
        # mnm = None
        optimizer.zero_grad()
        cat_preds, numeric_preds = model(
            num_inputs, cat_inputs, mnm=mnm, mlm=mlm, task="mlm"
        )
        cat_targets = torch.cat(
            (cat_inputs, model.numeric_indices.expand(cat_inputs.size(0), -1)), dim=1
        )
        cat_preds = cat_preds.permute(0, 2, 1)
        # print(
        #     f"cat_preds.shape: {cat_preds.shape}, cat_targets.shape: {cat_targets.shape}"
        # )
        cat_loss = ce_loss(cat_preds, cat_targets)
        numeric_loss = mse_loss(numeric_preds, num_inputs)
        loss = cat_loss + numeric_loss
        loss.backward()
        optimizer.step()
        batch_count += 1
        learning_rate = optimizer.param_groups[0]["lr"]
        summary_writer.add_scalar("Loss/masked_loss", loss.item(), batch_count)
        summary_writer.add_scalar("Loss/mlm_loss", cat_loss.item(), batch_count)
        summary_writer.add_scalar("Loss/mnm_loss", numeric_loss.item(), batch_count)
        summary_writer.add_scalar("Metrics/LearningRate", learning_rate, batch_count)
        if batch_count % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item():,.2f}")

Epoch 3/20 Loss: 0.71
Epoch 5/20 Loss: 0.31
Epoch 7/20 Loss: 0.14
Epoch 10/20 Loss: 0.09
Epoch 12/20 Loss: 0.09
Epoch 14/20 Loss: 0.08
Epoch 16/20 Loss: 0.07
Epoch 19/20 Loss: 0.06


Finetuning the previous model seems to work but when I pre-train, we run into issues. Let's try again


In [ ]:
# Regression Model

epochs = 20
batch_size = 1000
lr = 0.01
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

model_time = dt.now()
model_time = model_time.strftime("%Y-%m-%dT%H:%M:%S")
model_name = f"FullFineTune_Pre_train.8_{model_time}"

summary_writer = SummaryWriter("runs/" + model_name)


batch_count = 0
model.train()
for epoch in range(epochs):
    for i in range(0, X_train_num_tensor.size(0), batch_size):
        num_inputs = X_train_num_tensor[i : i + batch_size, :]
        cat_inputs = X_train_cat_tensor[i : i + batch_size, :]
        # mlm = torch.rand(cat_inputs.shape) > 0.8  # Greater than 1 for testing
        # mnm = torch.rand(num_inputs.shape) > 0.8
        mlm = None
        mnm = None
        optimizer.zero_grad()
        y_pred = model(num_inputs, cat_inputs, mnm=mnm, mlm=mlm)
        loss = loss_fn(y_pred, y_train_tensor[i : i + batch_size, :])
        loss.backward()
        optimizer.step()
        batch_count += 1
        learning_rate = optimizer.param_groups[0]["lr"]
        summary_writer.add_scalar("Loss/train", loss.item(), batch_count)
        summary_writer.add_scalar("Metrics/LearningRate", learning_rate, batch_count)
        if batch_count % 100 == 0:
            print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item():,.2f}")

Epoch 3/20 Loss: 2,084,017.88
Epoch 5/20 Loss: 1,750,660.25
Epoch 7/20 Loss: 2,160,727.25
Epoch 10/20 Loss: 1,608,132.50
Epoch 12/20 Loss: 2,133,934.00
Epoch 14/20 Loss: 2,029,316.12
Epoch 16/20 Loss: 1,611,975.88
Epoch 19/20 Loss: 1,844,371.62


In [ ]:
test_shape = X_train_num_tensor[i : i + batch_size, :].shape

In [ ]:
(torch.rand(test_shape) > 0.8).requires_grad

False

In [ ]:
# epochs = 40
# batch_size = 1000
# lr = 0.5
# loss_fn = nn.MSELoss()
# optimizer = optim.Adam(model.parameters(), lr=lr)
# batch_count = 0
# model.train()
# for epoch in range(epochs):
#     for i in range(0, X_train_num_tensor.size(0), batch_size):
#         optimizer.zero_grad()
#         y_pred = model(
#             X_train_num_tensor[i : i + batch_size, :],
#             X_train_cat_tensor[i : i + batch_size, :],
#         )
#         loss = loss_fn(y_pred, y_train_tensor[i : i + batch_size, :])
#         loss.backward()
#         optimizer.step()
#         batch_count += 1
#         if batch_count % 100 == 0:
#             print(f"Epoch {epoch+1}/{epochs} Loss: {loss.item():,.2f}")

In [ ]:
A = torch.zeros(4, 4, 4)
B = torch.ones(4, 4)

# Insert B into the slice where the first dimension is 2
A[2, :, :] = B

print(A)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])


In [ ]:
with torch.no_grad():
    y_pred = model(X_test_num_tensor[0:10, :], X_test_cat_tensor[0:10, :])
    loss = loss_fn(y_pred, y_test_tensor[0:10])
    print(f"Test loss: {loss.item():,.2f}")

Test loss: 2,266,821.75


In [ ]:
for i in range(10):
    print(
        f"Predicted: {y_pred[i].item():,.2f} Actual: {y_test_tensor[i].item():,.2f}",
        f"Diff: {y_pred[i].item() - y_test_tensor[i].item():,.2f}",
    )

Predicted: 32.68 Actual: 559.00 Diff: -526.32
Predicted: 2,149.53 Actual: 2,201.00 Diff: -51.47
Predicted: 973.39 Actual: 1,238.00 Diff: -264.61
Predicted: 1,424.54 Actual: 1,304.00 Diff: 120.54
Predicted: 10,648.70 Actual: 6,901.00 Diff: 3,747.70
Predicted: 3,545.51 Actual: 3,011.00 Diff: 534.51
Predicted: 957.34 Actual: 1,765.00 Diff: -807.66
Predicted: 1,408.64 Actual: 1,679.00 Diff: -270.36
Predicted: 1,732.66 Actual: 2,102.00 Diff: -369.34
Predicted: 7,455.69 Actual: 4,789.00 Diff: 2,666.69


```
Predicted: 2,085.14 Actual: 559.00 Diff: 1,526.14
Predicted: 3,381.02 Actual: 2,201.00 Diff: 1,180.02
Predicted: 1,725.17 Actual: 1,238.00 Diff: 487.17
Predicted: 1,914.37 Actual: 1,304.00 Diff: 610.37
Predicted: 15,271.41 Actual: 6,901.00 Diff: 8,370.41
Predicted: 7,173.91 Actual: 3,011.00 Diff: 4,162.91
Predicted: 947.38 Actual: 1,765.00 Diff: -817.62
Predicted: 604.91 Actual: 1,679.00 Diff: -1,074.09
Predicted: 989.06 Actual: 2,102.00 Diff: -1,112.94
Predicted: 9,508.21 Actual: 4,789.00 Diff: 4,719.21
```


In [ ]:
for i in range(10):
    print(f"Predicted: {y_pred[i].item():,.2f} Actual: {y_test_tensor[i].item():,.2f}")

Predicted: 32.68 Actual: 559.00
Predicted: 2,149.53 Actual: 2,201.00
Predicted: 973.39 Actual: 1,238.00
Predicted: 1,424.54 Actual: 1,304.00
Predicted: 10,648.70 Actual: 6,901.00
Predicted: 3,545.51 Actual: 3,011.00
Predicted: 957.34 Actual: 1,765.00
Predicted: 1,408.64 Actual: 1,679.00
Predicted: 1,732.66 Actual: 2,102.00
Predicted: 7,455.69 Actual: 4,789.00


In [ ]:
for i in range(10):
    print(f"Predicted: {y_pred[i].item():,.2f} Actual: {y_test_tensor[i].item():,.2f}")

Predicted: 32.68 Actual: 559.00
Predicted: 2,149.53 Actual: 2,201.00
Predicted: 973.39 Actual: 1,238.00
Predicted: 1,424.54 Actual: 1,304.00
Predicted: 10,648.70 Actual: 6,901.00
Predicted: 3,545.51 Actual: 3,011.00
Predicted: 957.34 Actual: 1,765.00
Predicted: 1,408.64 Actual: 1,679.00
Predicted: 1,732.66 Actual: 2,102.00
Predicted: 7,455.69 Actual: 4,789.00


In [ ]:
# Predicted: 688.47 Actual: 559.00
# Predicted: 2,547.17 Actual: 2,201.00
# Predicted: 1,044.95 Actual: 1,238.00
# Predicted: 1,790.95 Actual: 1,304.00
# Predicted: 10,160.45 Actual: 6,901.00
# Predicted: 3,790.90 Actual: 3,011.00
# Predicted: 1,729.77 Actual: 1,765.00
# Predicted: 1,749.63 Actual: 1,679.00
# Predicted: 2,328.92 Actual: 2,102.00
# Predicted: 5,928.75 Actual: 4,789.00

In [ ]:
torch.rand(3, 4, 5) > 0.8

tensor([[[False, False,  True, False, False],
         [ True, False, False, False, False],
         [False, False, False,  True, False],
         [False, False, False, False,  True]],

        [[False,  True, False,  True, False],
         [False,  True, False, False, False],
         [False, False,  True, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [ True, False,  True, False, False],
         [False, False, False, False,  True],
         [False, False,  True, False, False]]])

In [ ]:
torch.rand()

TypeError: rand() received an invalid combination of arguments - got (), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
